In [1]:
import sys
sys.path.append('..')
from model_building import HDRPointwiseNN
from dataset import HDRDataset
import os
from PIL import Image
import math

import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader,random_split
import pytorch_lightning as pl

from pytorch_lightning.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid


In [2]:
orig_photos_path = 'E:\Projects\StyleTransfer\justinho_dataset_processed\small_original_reduced_size'
target_photos_path = 'E:\Projects\StyleTransfer\justinho_dataset_processed\small_edited_reduced_size'

In [3]:
dataset = HDRDataset(orig_photos_path,target_photos_path)
dataset_length = len(dataset)
train_length = math.ceil(.8 *dataset_length)
val_length = dataset_length-train_length
train_set, val_set = random_split(dataset, [train_length, val_length], )#generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_set, batch_size = 4)
val_loader =  DataLoader(val_set, batch_size = 4)

skiplist is of length: 0
[]


In [4]:
print(dataset_length)
print(train_length)
print(val_length)


413
331
82


In [5]:
hdrnet = HDRPointwiseNN()

In [6]:
trainer = pl.Trainer(gpus=0,
                     max_epochs=5,
                     val_check_interval=.5,
                     terminate_on_nan=True)
trainer.fit(hdrnet, train_dataloader = train_loader, val_dataloaders = val_loader)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name            | Type                        | Params
----------------------------------------------------------------
0 | low_level       | LowLevelFeatures            | 24.8 K
1 | local_features  | LocalFeatures               | 74.1 K
2 | global_features | GlobalFeatures              | 377 K 
3 | fusion_layer    | FusionLayer                 | 0     
4 | pwc_mixing      | PointwiseChannelMixingLayer | 12.4 K
5 | reshape         | View                        | 0     
6 | guidance_layer  | GuidanceLayer               | 113   
7 | slicing_layer   | SlicingLayer                | 0     
8 | apply_coeffs    | ApplyCoeffs                 | 0     
----------------------------------------------------------------
489 K     Trainable params
0         Non-trainable params
489 K     Total params


fusion_output torch.Size([4, 128, 16, 16])
pwc_mix_output torch.Size([4, 96, 16, 16])
bilateral grid output torch.Size([1, 12, 8, 32, 32])
guidance output torch.Size([4, 1, 2048, 2048])


RuntimeError: grid_sampler(): expected grid and input to have same batch size, but got input with sizes [1, 12, 8, 32, 32] and grid with sizes [4, 1, 2048, 2048, 3]

In [ ]:
torch.cuda.is_available()

In [ ]:
!conda list pytorch

# Checking qualitatively

In [ ]:
model_path = r'E:\Projects\hdrnet-pytorch\lightning_logs\version_21\checkpoints\epoch=4-step=372.ckpt' 
hdrnet = HDRPointwiseNN()
with open(model_path,'rb') as f:
    checkpoint = torch.load(f, map_location=lambda storage, loc: storage)

hdrnet.load_state_dict(checkpoint['state_dict'])


In [ ]:
checkpoint.keys()


In [ ]:
checkpoint['callbacks'].values()

In [ ]:
input_image_reduced,input_image_full,output_image_full = next(iter(val_loader))
pred = hdrnet.forward(input_image_reduced,input_image_full,)

In [ ]:
f, axs = plt.subplots(pred.shape[0],3,figsize=(30,30))
for ii in range(pred.shape[0]):
    
    plt.subplot(pred.shape[0],3,3*ii+1)
    plt.imshow(input_image_full[ii].permute(1, 2, 0))
    
    plt.subplot(pred.shape[0],3,3*ii+2)
    plt.imshow(pred[ii].detach().permute(1, 2, 0))
    
    plt.subplot(pred.shape[0],3,3*ii+3)
    plt.imshow(output_image_full[ii].detach().permute(1, 2, 0))
    
    

In [ ]:
plt.imshow(pred_2.detach().permute(1, 2, 0))

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=/home/adong/hdrnet-pytorch/jho_logs/lightning_logs/version_1701084

In [ ]:
!pwd

In [ ]:
MyModel.load_from_checkpoint(checkpoint_path="example.ckpt")

In [ ]:
hdrnet = HDRPointwiseNN()

In [ ]:
hdrnet.load_from_checkpoint(checkpoint_path='/home/adong/hdrnet-pytorch/fivek_logs/lightning_logs/version_1494304/checkpoints/epoch=3-step=19991.ckpt')